Preparando o ambiente

In [ ]:
# Clonando pasta do github
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/tiagofiorini/MLinPhysics.git
    import os as os
    os.chdir('./MLinPhysics')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [ ]:
# Leitura do arquivo de dados
df = pd.read_excel('aula19_dados_serie_temporal.xlsx', sheet_name='dados')
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True) # dataframe indexado pela data
df.head()

# Arima Temperatura RMSP

Neste exercício vamos trabalhar apenas com a série temporal de temperatura.

In [ ]:
temp = df.temp
temp.plot()

Decomposição da série temporal em: tendência, sazonalidade e componente restante.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
tempdec = seasonal_decompose(temp, model='additive')
tempdec.plot()
plt.show()

Obtendo as funções de autocorrelação (ACF) e autocorrelação parcial (PACF)

In [ ]:
sm.graphics.tsa.plot_acf(temp, lags=40)
plt.show()

In [ ]:
sm.graphics.tsa.plot_pacf(temp, lags=40)
plt.show()

Vemos que a série temporal possui uma forte sazonalidade. Vamos fazer uma primeira tentativa de ajustar um modelo SARIMA aos dados de temperatura.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
# fit model
model1 = ARIMA(temp, order=(1,2,0))
model1_fit = model1.fit()
# summary of fit model
print(model1_fit.summary())
# line plot of residuals
residuals1 = pd.DataFrame(model1_fit.resid)
residuals1.plot(title='Gráfico dos resíduos')
plt.show()
# density plot of residuals
residuals1.plot(kind='kde', title='Distribuição dos resíduos')
plt.show()
# summary stats of residuals
print(residuals1.describe())
# Aplicando o modelo aos dados
pred1 = model1.fit().predict()
plt.plot(temp.index, temp, label='Observado')
plt.plot(pred1.index, pred1, label='Predito')
plt.ylabel('Temperatura (oC)')
plt.title('Modelo 1')
plt.legend()
plt.show()

Usando auto ARIMA para procurar o melhor modelo (determinação da ordem do modelo)

In [ ]:
# É preciso instalar a biblioteca autoarima (pmdarima) no colab:
!pip install statsmodels
!pip install pmdarima

In [ ]:
import statsmodels.api as sm
import pmdarima as pmd
autoarima=pmd.auto_arima(temp,start_p=1,start_q=0,m=12,seasonal=True,trace=True)
# m: number of periods in each season (ex: 12 for monthly data)
# seasonal: Whether to fit a seasonal ARIMA. Default is True.
# trace: Whether to print status on the fits.


In [ ]:
# fit model
model2=ARIMA(temp,order=(1,0,0),seasonal_order=(1,0,1,12))
model2_fit = model2.fit()
# summary of fit model
print(model2_fit.summary())
# line plot of residuals
residuals2 = pd.DataFrame(model2_fit.resid)
residuals2.plot(title='Gráfico dos resíduos')
plt.show()
# density plot of residuals
residuals2.plot(kind='kde', title='Distribuição dos resíduos')
plt.show()
# summary stats of residuals
print(residuals2.describe())
# Aplicando o modelo aos dados
pred2 = model2.fit().predict()
plt.plot(temp.index, temp, label='Observado')
plt.plot(pred2.index, pred2, label='Predito')
plt.ylabel('Temperatura (oC)')
plt.title('Modelo 2')
plt.legend()
plt.show()

Exercício: Ler a aba "exercicio" da mesma planilha, que uma série temporal de consumo de energia elétrica.

*   A série temporal possui tendência e sazonalidade?
*   Avalie as funções ACF e PACF e verifique se a série mostra evidências de processos de autocorrelação (AR) e de média móvel (MA).
*   Aplique um modelo da família ARIMA, escolhendo a ordem mais adequada




